In [43]:
import asyncio
from agno.agent import Agent
from agno.models.openai import OpenAIChat
from agno.tools.mcp import MCPTools, StreamableHTTPClientParams
import os
from dotenv import load_dotenv
load_dotenv()
os.environ['OPENAI_API_KEY']=os.getenv('OPENAI_API_KEY')
# 🔐 Replace with your actual Linear API key
LINEAR_API_KEY = os.getenv('LINEAR_API_KEY')

# 🌐 Hosted Linear MCP server endpoint
server_url = "https://mcp.linear.app/sse"

# ✅ Configure server parameters
server_params = StreamableHTTPClientParams(
    url="http://localhost:8000",
    headers={
        "Authorization": f"Bearer {LINEAR_API_KEY}"
    },
    # timeout=60,  # optional: request timeout in seconds
    # terminate_on_close=True  # optional: close the stream cleanly
)

try:
    async with MCPTools(server_params=server_params, transport="streamable-http") as mcp_tools:
        print("Loaded tools:", mcp_tools.tool_names)
        agent = Agent(model=OpenAIChat(id="gpt-4o"), tools=[mcp_tools])
        await agent.aprint_response("Create a new Linear issue titled 'App crashes on login'", stream=True)
except Exception as e:
    print("Error during agent run:", e)


CancelledError: Cancelled by cancel scope 21f5f9eacd0

In [8]:
import os
import asyncio
from dotenv import load_dotenv

from agno.agent import Agent
from agno.models.openai import OpenAIChat
from agno.tools.mcp import MCPTools, StreamableHTTPClientParams

# ✅ Load environment variables
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
LINEAR_API_KEY = os.getenv("LINEAR_API_KEY")

# ✅ Confirm keys are loaded
assert LINEAR_API_KEY and LINEAR_API_KEY.startswith("lin_api_"), "❌ Invalid or missing LINEAR_API_KEY"
assert os.environ["OPENAI_API_KEY"].startswith("sk-"), "❌ Invalid or missing OPENAI_API_KEY"

# ✅ Linear MCP endpoint
server_url = "https://mcp.linear.app/mcp"

# ✅ Setup MCP server params
server_params = StreamableHTTPClientParams(
    url=server_url,
    headers={"Authorization": f"Bearer {LINEAR_API_KEY}"},
    timeout=30,
    terminate_on_close=True
)

# ✅ Run the agent
async def run_agent():
    try:
        print("Connecting to Linear MCP...")
        async with MCPTools(server_params=server_params, transport="streamable-http") as mcp_tools:
            print("✅ Tools loaded:", mcp_tools.tool_names)

            agent = Agent(
                model=OpenAIChat(id="gpt-4o"),  # Use "gpt-4o" or "gpt-3.5-turbo"
                tools=[mcp_tools]
            )

            await agent.aprint_response(
                "Create a new Linear issue titled 'App crashes on login'",
                stream=True
            )
    except Exception as e:
        print("❌ Error:", type(e).__name__, str(e))

# ✅ Run (use await in Jupyter or asyncio.run in script)
await run_agent()



Connecting to Linear MCP...


CancelledError: Cancelled by cancel scope 1b601fe9f10

In [6]:
import os
import asyncio
from dotenv import load_dotenv

from agno.agent import Agent
from agno.models.openai import OpenAIChat
from agno.tools.mcp import MCPTools
from mcp.client.stdio import StdioServerParameters

# ✅ Load environment variables
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
LINEAR_API_KEY = os.getenv("LINEAR_API_KEY")

# ✅ Setup stdio command to run the Linear MCP proxy via npx
server_params = StdioServerParameters(
    command="npx",
    args=["-y", "mcp-remote", "https://mcp.linear.app/sse"],
    env={"LINEAR_API_KEY": LINEAR_API_KEY}
)

# ✅ Async agent runner
async def run_agent():
    try:
        async with MCPTools(server_params=server_params, transport="stdio") as mcp_tools:
            print("✅ Loaded tools:", mcp_tools.tool_names)

            agent = Agent(
                model=OpenAIChat(id="gpt-4o"),
                tools=[mcp_tools]
            )

            await agent.aprint_response("Create a new Linear issue titled 'App crashes on login'", stream=True)

    except Exception as e:
        print("❌ Error during agent run:", type(e).__name__, str(e))

# ✅ In Jupyter: use await; in .py file: use asyncio.run()
await run_agent()


❌ Error during agent run: UnsupportedOperation fileno


In [ ]:
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
import asyncio
from agno.agent import Agent
from agno.models.openai import OpenAIChat
from agno.tools.mcp import MCPTools, StreamableHTTPClientParams
import os
from dotenv import load_dotenv
load_dotenv()
os.environ['OPENAI_API_KEY']=os.getenv('OPENAI_API_KEY')
LINEAR_API_KEY = os.getenv('LINEAR_API_KEY')


async def create_mcp_agent(session):

    mcp_tools=MCPTools(session=session)
    await mcp_tools.initialize()
    return Agent(model=OpenAIChat(id="gpt-4o"), tools=[mcp_tools],instructions=["you are a Linear Task Management agent with Model Context Protocal tool of linear"],
                markdown=True, show_tool_calls=True)


async def runagent(message:str):
    server_params=StdioServerParameters(
        command="npx",
        args=["-y", "mcp-remote", "https://mcp.linear.app/sse"],
        env={"LINEAR_API_KEY": LINEAR_API_KEY}
    )
    try:
        async with stdio_client(server_params) as (read,write):
            async with ClientSession(read,write) as session:
                agent=await create_mcp_agent(session)
                await agent.aprint_response(message, stream="True")
    except Exception as e:
        print(f"Error {e}")
    finally:
        await asyncio.sleep(0.1)

if __name__=="__main__":
    try:
        asyncio.run(runagent("Create a new Linear issue titled 'App crashes on login'"))
    except Exception as e:
        print(f"Error {e}")



Error asyncio.run() cannot be called from a running event loop


C:\Users\sigma\AppData\Local\Temp\ipykernel_40164\328987272.py:36: RuntimeWarning: coroutine 'runagent' was never awaited
  print(f"Error {e}")


In [1]:
import os
import asyncio
import json
import time
from dotenv import load_dotenv
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from agno.agent import Agent
from agno.knowledge.pdf import PDFKnowledgeBase
from agno.vectordb.qdrant import Qdrant
from agno.storage.sqlite import SqliteStorage
from agno.models.openai import OpenAIChat
from agno.tools.mcp import MCPTools
from agno.exceptions import ModelProviderError

from io import StringIO
import sys
import logging

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

vector_db = Qdrant(
            collection="project_test2",
            url="http://localhost:6333"
        )
knowledge_base = PDFKnowledgeBase(
                path=f"data/user_abc",
                vector_db=vector_db
                )
knowledge_base.load(recreate=False)

INFO Embedder not provided, using OpenAIEmbedder as default.

INFO Creating collection

INFO Loading knowledge base

INFO Reading: Akreage Launch Pad Whitepaper

INFO Added 6 documents to knowledge base

INFO Reading: Akreage_Launchpad_Scope_of_Work (1)

INFO Added 2 documents to knowledge base

INFO Reading: Plan of Action

INFO Added 5 documents to knowledge base